In [1]:
import numpy as np
import pandas as pd
import requests
import json

key = '*******' #use your key

In [2]:
# https://cricapi.com/api/fantasySquad
# https://cricapi.com/api/playerFinder
# https://cricapi.com/api/cricketScore - live score
# https://cricapi.com/api/fantasySummary - final score card


In [3]:
uidd = {'2021-04-09': [1254058], '2021-04-10': [1254059], '2021-04-11': [1254060], '2021-04-12': [1254061], '2021-04-13': [1254062], '2021-04-14': [1254063], '2021-04-15': [1254064], '2021-04-16': [1254065], '2021-04-17': [1254066], '2021-04-18': [1254068, 1254067], '2021-04-19': [1254069], '2021-04-20': [1254070], '2021-04-21': [1254072, 1254071], '2021-04-22': [1254073], '2021-04-23': [1254074], '2021-04-24': [1254075], '2021-04-25': [1254077, 1254076], '2021-04-26': [1254078], '2021-04-27': [1254079], '2021-04-28': [1254080], '2021-04-29': [1254082, 1254081], '2021-04-30': [1254083], '2021-05-01': [1254084], '2021-05-02': [1254086, 1254085], '2021-05-03': [1254087], '2021-05-04': [1254088],
 '2021-05-05': [1254089], '2021-05-06': [1254090], '2021-05-07': [1254091], '2021-05-08': [1254093, 1254092], '2021-05-09': [1254095, 1254094], '2021-05-10': [1254096], '2021-05-11': [1254097], '2021-05-12': [1254098], '2021-05-13': [1254100, 1254099], '2021-05-14': [1254101], '2021-05-15': [1254102], '2021-05-16': [1254104, 1254103], '2021-05-17': [1254105], '2021-05-18': [1254106], '2021-05-19': [1254107], '2021-05-20': [1254108], '2021-05-21': [1254110, 1254109], '2021-05-22': [1254111], '2021-05-23': [1254113, 1254112]}

In [4]:
captains= {'422108': 2,'379143': 1.5, '308967': 2, '297433': 1.5, '931581': 2, '44936': 1.5, '793463': 2,'326016':2,
'422108': 2, '26421': 1.5, '34102': 2, '219889': 1.5, '1070173': 2, '439952': 1.5,'446507':2,'550215':1.5,'253802':1.5}

In [5]:
# Points per contribution
pts = {'R':1,'4s':1,'6s':2,'W':20,'M':20,'0s':1,'Cen':8,'Half-Cen':4,'4W':8,'5W':16,'catch':8,'stumped':12,'runout':6}
eco = {0:6, 5 : 4, 6 : 2, 7 : 0, 10:-2 , 11:-4, 12 : -6}

In [28]:
score = pd.read_excel('IPLScore2021.xlsx',sheet_name='Points',header = 0, index_col=0)
score.index = score.index.astype('str')

awards = pd.read_excel('IPLScore2021.xlsx',sheet_name='Awards',header = 0, index_col=0)
awards.index = awards.index.astype('str')




In [7]:
def upmr(ID, df2):
    '''Registers players who played current match and updates number of matches remaining'''
    MICSK = requests.get('https://cricapi.com/api/fantasySquad', params = {'apikey':key, 'unique_id':ID})
    mc = MICSK.json()
    
    for i in mc['squad']:
        for j in i['players']:
            if str(j['pid']) in df2.index:
                df2.loc[str(j['pid']),'ML'] -= 1
                
    #return mc            
             
            
    
    
    

In [8]:
def getpid(t1,t2):
    '''extracts list of player IDs of players involved in match'''
    pid = []
    for i in (t1+t2):
        pid.append(pidic.get(i))
    print(pid)    
    return pid    

In [9]:
def getscore(ID):
    MICSK = requests.get('https://cricapi.com/api/fantasySummary', params = {'apikey':key, 'unique_id':ID})
    mc = MICSK.json()
    
    return mc

In [10]:
def update(df2,df3, mc):
    for i in mc['data']['team']:
        for j in i['players']:
            if str(j['pid']) in df2.index:
                df2.loc[str(j['pid']),'total_score'] += 4*captains.get(str(j['pid']),1)
                df2.loc[str(j['pid']),'Round_'+str(14-df2.loc[str(j['pid']),'ML'])] += 4
            
            
            
    
    for i in mc['data']['fielding']:
        for j in i['scores']:
            if str(j['pid']) in df2.index:
                score = 0
                for k in ['stumped','catch','runout']:
                    score +=  pts[k]*j[k]
            
                df2.loc[str(j['pid']),'total_score'] += score*captains.get(str(j['pid']),1)
                
                df2.loc[str(j['pid']),'Round_'+str(14-df2.loc[str(j['pid']),'ML'])] += score
                df3.loc[str(j['pid']),'Catches'] += j['catch']
            
    for i in mc['data']['batting']:
        for j in i['scores']:
            if str(j['pid']) in df2.index:
                score = 0
                for k in ['R','4s','6s']:
                    score += pts[k]*j[k]
                
                if df2.loc[i,'Role'] != 'b':
                    if bat.loc['Runs'] == 0 and j['dismissal-info'] != 'not out':
                        score -= 2
           
                    
                if j['R'] >=100:
                    score += 16
                elif j['R'] >= 50:
                    score += 8
                elif j['R'] >= 30:
                    score += 4
                
                if df2.loc[str(j['pid']),'Role'] != 'b':
                    if j['B'] >=10:
                        if (j['SR'] <= 50): score -= 6
                        elif (j['SR'] <= 70): score -= 4
                        elif(j['SR'] <= 90): score -= 2
                        elif(j['SR'] > 180): score += 6
                        elif(j['SR'] > 150): score += 4
                        elif(j['SR'] > 130): score += 2 
                
                df2.loc[str(j['pid']),'total_score'] += score*captains.get(str(j['pid']),1)
                df2.loc[str(j['pid']),'Round_'+str(14-df2.loc[str(j['pid']),'ML'])] += score
                df3.loc[str(j['pid']),'Runs'] += j['R']
                df3.loc[str(j['pid']),'Sixes'] += j['6s']
                try:
                    if j['dismissal']== 'runout' and len(j['dismissal-by'])==1:
                        df2.loc[j['dismissal-by'][0]['pid'],'total_score'] += 6*captains.get(j['dismissal-by'][0]['pid'],1)
                        df2.loc[j['dismissal-by'][0]['pid'],'Round_'+str(14-df2.loc[j['dismissal-by'][0]['pid'],'ML'])] += 6
                except:
                    pass
                
                
                
                
                
    for i in mc['data']['bowling']:
        for j in i['scores']:
            if str(j['pid']) in df2.index:
                score = 0
                for k in ['M','W','0s']:
                    score += pts[k]*int(j.get(k,0))
                if int(j.get('W',0)) >= 5:
                    score += 16
                elif int(j.get('W',0)) == 4:
                    score += 8
                elif int(j.get('W',0)) == 3:
                    score += 4
                  
                
                try:
                    
                    if float(j['O']) >= 2:
                        for i in sorted(eco.keys(),reverse=True):
                                if float(j['Econ']) > i:
                                    score += eco[i]
                                    break
                except:
                    pass
                            
                   
                df2.loc[str(j['pid']),'total_score'] += score*captains.get(str(j['pid']),1)
                df2.loc[str(j['pid']),'Round_'+str(14-df2.loc[str(j['pid']),'ML'])] += score
                df3.loc[str(j['pid']),'Wickets'] += int(j.get('W',0))
                df3.loc[str(j['pid']),'Dots'] += int(j.get('0s',0))
                        
    
    

In [11]:
def gp(series):
    if sum(series != 0) ==0:
        return 0
    else:
        return sum(series)/sum(series != 0)

def pp(series):
    return sum(series != 0)

In [12]:
def gpp(df2):
    r =df2.groupby('Team')
    return r.total_score.sum()/(252 - r.ML.sum())

In [40]:
ID = uidd[pd.datetime.now().strftime('%Y-%m-%d')][0]

In [41]:
live = score.copy()
liveaw = awards.copy()
upmr(ID,live)

In [46]:
b = getscore(ID)

In [47]:
live['total_score'] = 0
liveaw = awards.copy()
#live['bowler'] = score.bowler
update(live,liveaw,b)

In [35]:
upmr(ID, score)
update(score,awards, b)

In [37]:
league = score.groupby('Team').agg(POINTS = ('total_score',np.sum)).sort_values('POINTS',ascending=False)
orange = awards.groupby('Team').agg(RUNS = ('Runs',np.sum)).sort_values('RUNS',ascending=False)
purple = awards.groupby('Team').agg(WICKETS = ('Wickets',np.sum)).sort_values('WICKETS',ascending=False)
dotted = awards.groupby('Team').agg(DOTS = ('Dots',np.sum)).sort_values('DOTS',ascending=False)
rocket = awards.groupby('Team').agg(SIXES = ('Sixes',np.sum)).sort_values('SIXES',ascending=False)
bucket = awards.groupby('Team').agg(CATCHES = ('Catches',np.sum)).sort_values('CATCHES',ascending=False)

In [1]:


with pd.ExcelWriter('IPLScore2021.xlsx') as ex:
    score.to_excel(ex, sheet_name='Points')
    awards.to_excel(ex, sheet_name='Awards')



In [115]:
score.to_csv('IPL2021/score.csv',index=False,columns=score.columns[:-1])
awards.to_csv('IPL2021/awards.csv',index=False)